In [1]:
from __future__ import print_function

import numpy as np
import pandas as pd
import recordlinkage as rl
import pickle
from recordlinkage.index import Block
from recordlinkage.index import Full
from recordlinkage.index import SortedNeighbourhood
from recordlinkage.compare import Exact, String
from recordlinkage.datasets import load_febrl4

In [39]:
#Link File Field Variables
LF_CN = 'Organisation Name' #Company Name Field
LF_CNU = 'Company/Charity Number'
LF_A1 = 'Address Line 1'
LF_A2 = 'Address Line 2'
LF_CITY = 'City'
LF_COUNTY = 'State/Province'
LF_PC = 'Postal Code'

In [4]:
df_LW = pd.read_csv(r"C:\Users\jacks\Documents\Document(Offline)\Barcanet\Record Linkage\RLW-Accreditations.csv",encoding = "ISO-8859-1" ) 
df_BCD = pd.read_csv(r"C:\Users\jacks\Documents\Document(Offline)\BCD.csv",encoding = "ISO-8859-1",low_memory=False) 


In [40]:
df_LW = pd.read_csv(r"C:\Users\jacks\Documents\Document(Offline)\Barcanet\Record Linkage\RLW-Accreditations.csv",encoding = "ISO-8859-1" ) 

df_LW

,Organisation Name,Company/Charity Number,Address Line 1,Address Line 2,City,State/Province,Postal Code
0,10:10,NaN,8 Delancey Passage,NaN,London,Greater London,NW1 7NN
1,0800 WASTE LTD,NaN,21 HASLAM AVE,NaN,SM39ND,NaN,SM3 9ND
2,100%Open,NaN,1 Mercers Manor Barns,NaN,"Sherington, Newport Pagnel",Buckinghamshire,WC2B 4HN
3,108 The Bakery,NaN,"Unit 7, Basepoint",Premier Way,Romsey,NaN,SO51 9AQ
4,11:FS,NaN,2nd Floor,1 Finsbury Avenue,London,NaN,EC2M 2PF
...,...,...,...,...,...,...,...
7775,Zetetick - a unique housing charity,NaN,"Innovation Centre, Highfield Drive",NaN,St Leonards-on-Sea,NaN,TN38 9UH
7776,Zoocha,NaN,"Second Floor, Seed Warehouse, The Wash",NaN,Hertford,NaN,SG14 1PX
7777,Zurich Insurance plc,NaN,70 Mark Lane,NaN,London,NaN,EC3R 7NQ
7778,Zymurgorium,NaN,"Unit B6 Fairhills Trading Centre,",NaN,"Irlam, Greater manchester",NaN,M44 6BA


In [5]:
df_BCD = df_BCD.drop(df_BCD.columns[10:], axis=1)
df_BCD = df_BCD.drop(['RegAddress#CareOf',
                      'RegAddress#POBox',
                     'RegAddress#Country'], axis=1)
df_BCD

,CompanyName,CompanyNumber,RegAddress#AddressLine1,RegAddress#AddressLine2,RegAddress#PostTown,RegAddress#County,RegAddress#PostCode
0,! LTD,8209948,METROHOUSE 57 PEPPER ROAD,HUNSLET,LEEDS,YORKSHIRE,LS10 2RU
1,!? LTD,11399177,THE STUDIO HATHERLOW HOUSE,HATHERLOW,ROMILEY,NaN,SK6 3DY
2,!BIG IMPACT GRAPHICS LIMITED,11743365,372 OLD STREET,335 ROSDEN HOUSE,LONDON,NaN,EC1V 9LT
3,!L PRODUCTIONS LIMITED,12402527,95 MARMION AVENUE,NaN,CHINGFORD,NaN,E4 8EJ
4,!NKED LTD,12234705,29 CORRY DRIVE,NaN,LONDON,NaN,SW9 8QS
...,...,...,...,...,...,...,...
4506996,JASJ LIMITED,SC459332,5 CARDEN PLACE,NaN,ABERDEEN,NaN,AB10 1UT
4506997,JASJ PARTNERSHIP LLP,OC427693,20 LEADALE AVENUE,NaN,LONDON,NaN,E4 8AT
4506998,JASJACK CYCLES LTD,7763816,5 CROSS CLOSE,FREMINGTON,BARNSTAPLE,NaN,EX31 3AN
4506999,JASJEEV LIMITED,6927959,18 ALUM CLOSE,NaN,COVENTRY,WEST MIDLANDS,CV6 5TQ


In [41]:
normalised_names = []
to_remove = ['\([^()]*\)','ltd','Ltd','LTD', 'Limited', 'limited','LIMITED',' Group','GROUP','Holdings','HOLDINGS','LLP']
for i in df_LW[LF_CN]:
    for j in to_remove:
        i = i.replace(j, '')
    i.rstrip()
    normalised_names.append(i)
df_LW['Normalised Organisation Name'] = normalised_names
del df_LW[LF_CN]
df_LW

,Company/Charity Number,Address Line 1,Address Line 2,City,State/Province,Postal Code,Normalised Organisation Name
0,NaN,8 Delancey Passage,NaN,London,Greater London,NW1 7NN,10:10
1,NaN,21 HASLAM AVE,NaN,SM39ND,NaN,SM3 9ND,0800 WASTE
2,NaN,1 Mercers Manor Barns,NaN,"Sherington, Newport Pagnel",Buckinghamshire,WC2B 4HN,100%Open
3,NaN,"Unit 7, Basepoint",Premier Way,Romsey,NaN,SO51 9AQ,108 The Bakery
4,NaN,2nd Floor,1 Finsbury Avenue,London,NaN,EC2M 2PF,11:FS
...,...,...,...,...,...,...,...
7775,NaN,"Innovation Centre, Highfield Drive",NaN,St Leonards-on-Sea,NaN,TN38 9UH,Zetetick - a unique housing charity
7776,NaN,"Second Floor, Seed Warehouse, The Wash",NaN,Hertford,NaN,SG14 1PX,Zoocha
7777,NaN,70 Mark Lane,NaN,London,NaN,EC3R 7NQ,Zurich Insurance plc
7778,NaN,"Unit B6 Fairhills Trading Centre,",NaN,"Irlam, Greater manchester",NaN,M44 6BA,Zymurgorium


In [10]:
normalised_names = []
to_remove = ['\([^()]*\)','ltd','Ltd','LTD', 'Limited', 'limited','LIMITED',' Group','GROUP','Holdings','HOLDINGS','LLP']
for i in df_BCD['CompanyName']:
    for j in to_remove:
        i = i.replace(j, '')
    i.rstrip()
    normalised_names.append(i)
df_BCD['Normalised Names'] = normalised_names
del df_BCD['CompanyName']
df_BCD

,CompanyNumber,RegAddress#AddressLine1,RegAddress#AddressLine2,RegAddress#PostTown,RegAddress#County,RegAddress#PostCode,Normalised Names
0,8209948,METROHOUSE 57 PEPPER ROAD,HUNSLET,LEEDS,YORKSHIRE,LS10 2RU,!
1,11399177,THE STUDIO HATHERLOW HOUSE,HATHERLOW,ROMILEY,NaN,SK6 3DY,!?
2,11743365,372 OLD STREET,335 ROSDEN HOUSE,LONDON,NaN,EC1V 9LT,!BIG IMPACT GRAPHICS
3,12402527,95 MARMION AVENUE,NaN,CHINGFORD,NaN,E4 8EJ,!L PRODUCTIONS
4,12234705,29 CORRY DRIVE,NaN,LONDON,NaN,SW9 8QS,!NKED
...,...,...,...,...,...,...,...
4506996,SC459332,5 CARDEN PLACE,NaN,ABERDEEN,NaN,AB10 1UT,JASJ
4506997,OC427693,20 LEADALE AVENUE,NaN,LONDON,NaN,E4 8AT,JASJ PARTNERSHIP
4506998,7763816,5 CROSS CLOSE,FREMINGTON,BARNSTAPLE,NaN,EX31 3AN,JASJACK CYCLES
4506999,6927959,18 ALUM CLOSE,NaN,COVENTRY,WEST MIDLANDS,CV6 5TQ,JASJEEV


In [42]:
#print(df_LW)
#print(df_LW[LF_A2].str.upper())
df_LW['Normalised Organisation Name'] = df_LW['Normalised Organisation Name'].str.upper()
df_LW[LF_A1] = df_LW[LF_A1].str.upper()
df_LW[LF_A2] = df_LW[LF_A2].str.upper()
df_LW[LF_CITY] = df_LW[LF_CITY].str.upper()
df_LW[LF_COUNTY] = df_LW[LF_COUNTY].str.upper()
df_BCD['Normalised Names'] = df_BCD['Normalised Names'].str.upper()

0                     NaN
1                     NaN
2                     NaN
3             PREMIER WAY
4       1 FINSBURY AVENUE
              ...        
7775                  NaN
7776                  NaN
7777                  NaN
7778                  NaN
7779                  NaN
Name: Address Line 2, Length: 7780, dtype: object


In [43]:
print(len(df_LW))
df_LW = df_LW.reset_index()
df_BCD = df_BCD.reset_index()
print(len(df_LW))
df_LW

7780
7780


,index,Company/Charity Number,Address Line 1,Address Line 2,City,State/Province,Postal Code,Normalised Organisation Name
0,0,NaN,8 DELANCEY PASSAGE,NaN,LONDON,GREATER LONDON,NW1 7NN,10:10
1,1,NaN,21 HASLAM AVE,NaN,SM39ND,NaN,SM3 9ND,0800 WASTE
2,2,NaN,1 MERCERS MANOR BARNS,NaN,"SHERINGTON, NEWPORT PAGNEL",BUCKINGHAMSHIRE,WC2B 4HN,100%OPEN
3,3,NaN,"UNIT 7, BASEPOINT",PREMIER WAY,ROMSEY,NaN,SO51 9AQ,108 THE BAKERY
4,4,NaN,2ND FLOOR,1 FINSBURY AVENUE,LONDON,NaN,EC2M 2PF,11:FS
...,...,...,...,...,...,...,...,...
7775,7775,NaN,"INNOVATION CENTRE, HIGHFIELD DRIVE",NaN,ST LEONARDS-ON-SEA,NaN,TN38 9UH,ZETETICK - A UNIQUE HOUSING CHARITY
7776,7776,NaN,"SECOND FLOOR, SEED WAREHOUSE, THE WASH",NaN,HERTFORD,NaN,SG14 1PX,ZOOCHA
7777,7777,NaN,70 MARK LANE,NaN,LONDON,NaN,EC3R 7NQ,ZURICH INSURANCE PLC
7778,7778,NaN,"UNIT B6 FAIRHILLS TRADING CENTRE,",NaN,"IRLAM, GREATER MANCHESTER",NaN,M44 6BA,ZYMURGORIUM


In [44]:
indexer = rl.Index()

indexer.add(SortedNeighbourhood(left_on='Normalised Organisation Name', right_on='Normalised Names',window=3))

candidate_links = indexer.index(df_LW, df_BCD)

candidate_links

MultiIndex([(   0,    6084),
            (   1,    1849),
            (   2,    4767),
            (   3,    5937),
            (   4,    7919),
            (   5,    7962),
            (   6,    8794),
            (   7,    9171),
            (   8,    9264),
            (   9,   13432),
            ...
            (7770, 4096025),
            (7771, 4096062),
            (7772, 4096065),
            (7773, 4096070),
            (7774, 4096597),
            (7775, 4096992),
            (7776, 4103386),
            (7777, 4105844),
            (7778, 4106591),
            (7779, 4106855)],
           length=18223)

In [68]:
comparer = rl.Compare()
comparer.add(String('Normalised Organisation Name', 'Normalised Names', method='jarowinkler',label='SIM Organisation Name'))#pass
comparer.add(String('Company/Charity Number', 'CompanyNumber', method='jarowinkler',label='SIM CompanyNumber'))
comparer.add(String(LF_A1, 'RegAddress#AddressLine1', method='jarowinkler',label='SIM Address Line 1'))
comparer.add(String(LF_A2, 'RegAddress#AddressLine2', method='jarowinkler',label='SIM Address Line 2'))
comparer.add(Exact(LF_CITY, 'RegAddress#PostTown', label='City'))
comparer.add(Exact(LF_COUNTY, 'RegAddress#County', label='County'))
comparer.add(Exact('Postal Code', 'RegAddress#PostCode', label='Post Code'))#fail


features = comparer.compute(candidate_links, df_LW, df_BCD)
print('feature shape', features.shape)

feature shape (18223, 7)


In [75]:
features

,,SIM Organisation Name,SIM CompanyNumber,SIM Address Line 1,SIM Address Line 2,City,County,Post Code
0,6084,0.770833,0.0,0.670085,0.000000,0,0,0
1,1849,0.972222,0.0,0.937500,0.000000,0,0,1
2,4767,0.805556,0.0,0.569805,0.000000,0,0,0
3,5937,0.778571,0.0,0.471678,0.000000,0,0,0
4,7919,0.785714,0.0,0.476580,0.483287,1,0,1
...,...,...,...,...,...,...,...,...
7775,4096992,0.829206,0.0,0.473420,0.000000,0,0,0
7776,4103386,0.688889,0.0,0.537779,0.000000,0,0,0
7777,4105844,0.898224,0.0,0.503595,0.000000,0,0,0
7778,4106591,0.603367,0.0,0.479618,0.000000,0,0,0


In [73]:
from sklearn import svm


with open(r'C:\Users\jacks\Documents\GitHub\Record-Linkage\Models\RL_M6_SVM.pkl', 'rb') as file:
    model = pickle.load(file)
output = model.predict(features)


c:\Users\jacks\env\lib\site-packages\sklearn\base.py:315: UserWarning: Trying to unpickle estimator LinearSVC from version 0.22.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


ValueError: X has 7 features per sample; expecting 1

In [18]:
features_columns = ['SIM Organisation Name','SIM CompanyNumber','SIM Address Line 1','SIM Address Line 2','Exact City','Exact County','Exact Post Code']
LW_columns = (df_LW.columns.values)
BCD_columns = (df_BCD.columns.values)
combined_columns = np.concatenate([features_columns,LW_columns,BCD_columns])

df_combined = []
j=1
df_combined = pd.DataFrame(columns = combined_columns)


for i, row in features.iterrows():
    LW_list = df_LW.iloc[(i[0])].tolist()
    BCD_list = df_BCD.iloc[(i[1])].tolist()
    Combined_list = row.tolist() + LW_list + BCD_list
    df_combined.loc[len(df_combined)] = Combined_list
    print(j)
    '''if j ==5:
        break'''
    j+=1
df_combined

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


,SIM Organisation Name,SIM CompanyNumber,SIM Address Line 1,SIM Address Line 2,Exact City,Exact County,Exact Post Code,index,Organisation ID,Company/Charity Number,...,Stage,Normalised Organisation Name,index,CompanyNumber,RegAddress#AddressLine1,RegAddress#AddressLine2,RegAddress#PostTown,RegAddress#County,RegAddress#PostCode,Normalised Names
0,0.667125,0.428571,0.000000,0.000000,0.0,0.0,0.0,0,0012I00002ffKqr,4612928,...,Accredited (paid),4D CARE,48928,9993415,HATTERS FARM STOTT LANE,MIDDLETON,MANCHESTER,NaN,M24 6XL,4D CLEANING & MAINTENANCE
1,0.860358,0.666667,0.384722,0.000000,0.0,0.0,0.0,1,0012I00002gQW0q,SC233625,...,Accredited (paid),ABERDEEN HEAT AND POWER,115213,12053836,STUDIO 210,134-146 CURTAIN ROAD,LONDON,NaN,EC2A 3AR,ABERDEEN HELPING HANDS
2,0.876461,0.416667,0.502976,0.000000,0.0,0.0,0.0,2,0012I00002Zbd0n,sc500055,...,Accredited (paid),ABODE CAFE BAR,119422,11230639,4 FRANCIS STREET,NaN,LEICESTER,NaN,LE2 2BD,ABODE CARPENTRY
3,0.944561,0.523810,0.584641,0.000000,0.0,0.0,0.0,3,0012I00002gRCB9,6995515,...,Accredited (paid),ABSOLUTE WORKSPACE,122775,8803575,32A EAST STREET,NaN,ST. IVES,NaN,PE27 5PD,ABSOLUTE WRAPS
4,0.933333,0.511905,0.491588,0.000000,0.0,0.0,0.0,4,0012I00002cyIBk,1055187,...,Accredited (paid),ACE OF CLUBS,133340,12493052,"C/O WOODSTOCK ACCOUNTANCY, 3A",MARKET PLACE,WOODSTOCK,NaN,OX20 1SY,ACE OF CUPS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2147,0.638095,0.000000,0.433532,0.000000,0.0,0.0,0.0,251,0010G00002Jbw2v,NaN,...,Accreditation Closed,TREES HOME HELP,3758468,12179761,TINTAGEL ST. CLAIRS ROAD,ST. OSYTH,CLACTON-ON-SEA,NaN,CO16 8QG,TREES FOR GUNS C.I.C.
2148,0.847619,0.000000,0.554449,0.445221,0.0,0.0,0.0,252,0012I00002YTKx5,NaN,...,Accreditation Closed,TREZEO,3762341,5425817,THE TECHNOLOGY CENTRE STATION ROAD,FRAMLINGHAM,WOODBRIDGE,NaN,IP13 9EZ,TREZAK
2149,0.878804,0.000000,0.506373,0.000000,0.0,0.0,0.0,253,001F000001QreYs,NaN,...,Accreditation Closed,UPSTAIRS DOWNSTAIRS HOUSEKEEPING,3836624,4737192,DEVONSHIRE HOUSE,582 HONEYPOT LANE,STANMORE,MIDDLESEX,HA7 1JS,UPSTAIRS DOWNSTAIRS CONTRACTORS
2150,0.661552,0.000000,0.472767,0.000000,0.0,0.0,0.0,254,0012I00002aROuA,NaN,...,Accreditation Closed,VIRTUE GLOBAL,3893836,10344023,OFFICE 7 35-37 LUDGATE HILL,NaN,LONDON,NaN,EC4M 7JN,VIRTUE FORESTRY MANAGEMENT


In [19]:
df_combined.to_csv(r'C:\Users\Clamfighter\Machine_Learning_Project\my_env\Heggle\Record Linkage - Company Data\Company Data\Outputs\RLW_JUNE_Data.csv',index=False)